## Simplifying data with the singular value decomposition

- Pros: Simplifies data, removes noise, may improve algorithm results.
- Cons: Transformed data may be difficult to understand.
- Works with: Numeric values.

In [1]:
import numpy as np

### Experiment 1: Toy dataset

In [2]:
def loadExData():
    return [[0, 0, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1],
            [1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 1, 0, 0]]

In [3]:
Data = loadExData()
U, Sigma, VT = np.linalg.svd(Data)
print(Sigma)

[9.64365076e+00 5.29150262e+00 7.40623935e-16 4.05103551e-16
 2.21838243e-32]


In [4]:
Sig3 = np.mat([[Sigma[0], 0, 0], [0, Sigma[1], 0], [0, 0, Sigma[2]]])
with np.printoptions(precision=2):
    print(U[:, :3] * Sig3 * VT[:3, :])

[[ 5.03e-17  1.95e-15  1.71e-15  2.00e+00  2.00e+00]
 [-7.69e-16  3.15e-16  4.55e-16  3.00e+00  3.00e+00]
 [-2.02e-16  6.40e-17  1.38e-16  1.00e+00  1.00e+00]
 [ 1.00e+00  1.00e+00  1.00e+00 -1.52e-33 -1.22e-33]
 [ 2.00e+00  2.00e+00  2.00e+00 -3.04e-33 -2.43e-33]
 [ 5.00e+00  5.00e+00  5.00e+00  1.82e-33  1.46e-33]
 [ 1.00e+00  1.00e+00  1.00e+00 -1.52e-33 -1.22e-33]]


### Experiment 2: Recommendation system

In [5]:
# Similarity measures
def ecludSim(inA, inB):
    return 1.0 / (1.0 + np.linalg.norm(inA - inB))


def pearsSim(inA, inB):
    if len(inA) < 3:
        return 1.0
    return 0.5 + 0.5 * np.corrcoef(inA, inB, rowvar=False)[0][1]


def cosSim(inA, inB):
    num = float(inA.T * inB)
    denom = np.linalg.norm(inA) * np.linalg.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [6]:
# Item-based recommendation engine
def standEst(dataMat, user, simMeas, item):
    n = np.shape(dataMat)[1]
    simTotal, ratSimTotal = 0.0, 0.0
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        # Find users who rated both items
        overLap = np.nonzero(np.logical_and(dataMat[:, item].A > 0,
                                            dataMat[:, j].A > 0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        # print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal


def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    # Find unrated items
    unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
    if len(unratedItems) == 0:
        return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    # Return top N unrated items
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [7]:
myMat = np.mat(loadExData())
myMat[0, 1] = myMat[0, 0] = myMat[1, 0] = myMat[2, 0] = 4
myMat[3, 3] = 2
print(myMat)

[[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]


In [8]:
recommend(myMat, 2)

[(2, 2.5), (1, 2.0243290220056256)]

In [9]:
recommend(myMat, 2, simMeas=ecludSim)

[(2, 3.0), (1, 2.8266504712098603)]

In [10]:
recommend(myMat, 2, simMeas=pearsSim)

[(2, 2.5), (1, 2.0)]

In [11]:
# Rating estimation by using the SVD
def svdEst(dataMat, user, simMeas, item):
    n = np.shape(dataMat)[1]
    simTotal, ratSimTotal = 0.0, 0.0
    U, Sigma, VT = np.linalg.svd(dataMat)
    # Create diagonal matrix
    Sig3 = np.mat(np.eye(4) * Sigma[:4])
    # Create transformed items
    xformedItems = dataMat.T * U[:, :4] * Sig3.I
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal

In [12]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [13]:
myMat = np.mat(loadExData2())

In [14]:
U, Sigma, VT = np.linalg.svd(myMat)
Sig2 = Sigma ** 2

In [15]:
print(sum(Sig2) * 0.9)
print(sum(Sig2[:2]))
print(sum(Sig2[:3]))

487.7999999999996
378.8295595113579
500.50028912757926


In [16]:
recommend(myMat, 1, estMethod=svdEst)

[(4, 3.344714938469228), (7, 3.329402072452697), (9, 3.328100876390069)]

In [17]:
recommend(myMat, 1, estMethod=svdEst, simMeas=pearsSim)

[(4, 3.346952186702173), (9, 3.3353796573274694), (6, 3.3071930278130366)]

### Experiment 3: Image compression

In [18]:
# Image-compression functions
def printMat(inMat, thresh=0.8):
    for i in range(32):
        print((inMat.A[i, :] > thresh).astype(int))


def imgCompress(numSV=3, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = np.mat(myl)
    print("****original matrix******")
    printMat(myMat, thresh)
    U, Sigma, VT = np.linalg.svd(myMat)
    SigRecon = np.mat(np.zeros((numSV, numSV)))
    for k in range(numSV):
        SigRecon[k, k] = Sigma[k]
    reconMat = U[:, :numSV] * SigRecon * VT[:numSV, :]
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [19]:
imgCompress(2)

****original matrix******
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1